# 데이터 크롤링

In [ ]:
## 1. 패키지 불러오기
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import re

table = pd.DataFrame()

In [ ]:
# 일반 영상
def general_video(url):
    global table
    
    driver.get(url)
    time.sleep(5)

    results = []
    
    # 제목
    title = driver.find_element(By.XPATH, '//*[@id="watch7-content"]/meta[1]').get_attribute('content')

    # 댓글 쪽으로 스크롤하기
    driver.execute_script("window.scrollTo(0, 500)")
    time.sleep(3)

    # 더보기 버튼 클릭
    sort_button = driver.find_element(By.XPATH, '//*[@id="expand"]')
    sort_button.click()
    time.sleep(1)
    
    # published 날짜
    date = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]').text

    # 댓글 개수
    count_comment_element = driver.find_element(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[1]')
    count_comment = count_comment_element.text

    # 댓글 정렬 버튼 클릭
    sort_button = driver.find_element(By.XPATH, '//*[@id="icon-label"]')
    sort_button.click()
    time.sleep(1)

    # 인기순 클릭
    popular_option = driver.find_element(By.XPATH, '//*[@id="item-with-badge"]/div')
    popular_option.click()
    time.sleep(2)

    # 댓글 데이터 수집
    comments = []
    likes_counts = []  # 댓글 좋아요 수
    comment_dates = []  # 댓글 작성 날짜
    comment_sub_counts = []  # 대댓글 수 저장 리스트

    # 페이지 로딩
    for _ in range(30):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)  # 페이지 로딩 대기

    # 파싱할 데이터 불러오기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')

    # 댓글 정보 수집
    comment_list = soup.select("ytd-comment-thread-renderer")
    for comment in comment_list:
        if len(comments) < 500:
            # 댓글 내용
            temp_comment = comment.select_one("#content-text").text.strip()
            comments.append(temp_comment)

            # 댓글 좋아요 수
            like_count = comment.select_one("#vote-count-middle").text.strip() if comment.select_one("#vote-count-middle") else '0'
            likes_counts.append(like_count)
                    
            # 댓글 작성 날짜
            comment_date_element = comment.select_one("#published-time-text a")
            comment_date = comment_date_element.text.strip() if comment_date_element else 'N/A'
            comment_dates.append(comment_date)

            # 대댓글 수 추출
            sub_comment_count_element = comment.select_one("button[aria-label*='reply'], button[aria-label*='replies']")
            if sub_comment_count_element:
                sub_comment_count_text = sub_comment_count_element.get('aria-label', '').strip()
                # 숫자만 추출하는 정규 표현식 사용
                
                match = re.search(r'\d+', sub_comment_count_text)
                
                if match:
                    comment_sub_counts.append(int(match.group(0)))  # 추출한 숫자를 정수로 변환하여 추가
                else:
                    comment_sub_counts.append(0)  # 숫자가 없으면 0으로
            else:
                comment_sub_counts.append(0)  # 대댓글이 없으면 0으로

    # 영상 제목, 영상 published 날짜, 댓글 수, 인기 댓글 내용, 댓글 좋아요 수, 대댓글 수 전부 한 곳에 모으기
    for comment in comments:
        results.append({
            'title': title,
            'date': date,
            'count_comment': count_comment,
            'comment': comment,
            'likes': likes_counts.pop(0) if likes_counts else '0',  # likes_counts에서 첫 번째 요소 제거
            'comment_date': comment_dates.pop(0) if comment_dates else 'N/A',  # comment_dates에서 첫 번째 요소 제거
            'sub_comment_count': comment_sub_counts.pop(0) if comment_sub_counts else 0  # sub_comment_counts에서 첫 번째 요소 제거
        })
    
    # 데이터프레임으로 변환
    table = table.append(pd.DataFrame(results), ignore_index=True)
    print(table)


In [ ]:
# 쇼츠
def short_video(url):
    global table
    
    driver.get(url)
    time.sleep(5)

    results = []
    
    # 제목
    title = driver.find_element(By.XPATH, '//*[@id="watch7-content"]/meta[1]').get_attribute('content')

    # published 날짜 추출 개선
    date_elements = driver.find_elements(By.XPATH, '//factoid-renderer//div[@class="ytwFactoidRendererFactoid"]')

    # 날짜 정보 추출
    date = ''
    aria_label = date_elements[2].get_attribute('aria-label')  # aria-label에서 날짜 추출
    if aria_label:  # aria-label이 존재하는 경우
        date = aria_label.strip()  # 날짜 값을 가져옴

    # 댓글 클릭하기
    comment_button = driver.find_element(By.XPATH, '//*[@id="comments-button"]/ytd-button-renderer/yt-button-shape/label/button/yt-touch-feedback-shape/div')
    comment_button.click()
    time.sleep(3)

    # 댓글 개수
    count_comment_element = driver.find_element(By.XPATH, '//*[@id="comments-button"]/ytd-button-renderer/yt-button-shape/label/div/span')
    count_comment = count_comment_element.text

    # 댓글 정렬 버튼 클릭
    sort_button = driver.find_element(By.XPATH, '//*[@id="label-icon"]')
    sort_button.click()
    time.sleep(1)

    # 인기순 클릭
    popular_option = driver.find_element(By.XPATH, '//*[@id="menu"]/a[1]/tp-yt-paper-item/tp-yt-paper-item-body')
    popular_option.click()
    time.sleep(2)
    
    # 댓글 정렬 버튼 한번 더 클릭 
    sort_button = driver.find_element(By.XPATH, '//*[@id="label-icon"]')
    sort_button.click()
    time.sleep(1)
    
    # 댓글 데이터 수집
    comments = []
    likes_counts = []  # 댓글 좋아요 수
    comment_dates = []  # 댓글 작성 날짜
    comment_sub_counts = []  # 대댓글 수 저장 리스트

    # 댓글 영역 선택
    reply_body = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[4]/div[2]/ytd-engagement-panel-section-list-renderer[1]/div[2]/ytd-section-list-renderer/div[2]/ytd-comments/ytd-item-section-renderer/div[3]')
     
    #댓글 스크롤
    for _ in range(40):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight;', reply_body)
        time.sleep(2)  # 페이지 로딩 대기
           
    # 파싱할 데이터 불러오기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
        
    # 댓글 정보 수집
    comment_list = soup.select("ytd-comment-thread-renderer")
    for comment in comment_list:
        if len(comments) < 500:
            # 댓글 내용
            temp_comment = comment.select_one("#content-text").text.strip()
            comments.append(temp_comment)

            # 댓글 좋아요 수
            like_count = comment.select_one("#vote-count-middle").text.strip() if comment.select_one("#vote-count-middle") else '0'
            likes_counts.append(like_count)

            # 댓글 작성 날짜
            comment_date_element = comment.select_one("#published-time-text a")
            comment_date = comment_date_element.text.strip() if comment_date_element else 'N/A'
            comment_dates.append(comment_date)
                    
            # 대댓글 수 추출
            sub_comment_count_element = comment.select_one("button[aria-label*='reply'], button[aria-label*='replies']")
            if sub_comment_count_element:
                sub_comment_count_text = sub_comment_count_element.get('aria-label', '').strip()
                comment_sub_counts.append(int(sub_comment_count_text.split(' ')[0]))
            else:
                comment_sub_counts.append(0)  # 대댓글이 없으면 0으로
                
        else:
            break
        

    # 영상 제목, 영상 published 날짜, 댓글 수, 인기 댓글 내용, 댓글 좋아요 수, 대댓글 수 전부 한 곳에 모으기
    for comment in comments:
        results.append({
            'title': title,
            'date': date,
            'count_comment': count_comment,
            'comment': comment,
            'likes': likes_counts.pop(0) if likes_counts else '0',  # likes_counts에서 첫 번째 요소 제거
            'comment_date': comment_dates.pop(0) if comment_dates else 'N/A',  # comment_dates에서 첫 번째 요소 제거
            'sub_comment_count': comment_sub_counts.pop(0) if comment_sub_counts else 0  # sub_comment_counts에서 첫 번째 요소 제거
        })
    
    # 데이터프레임으로 변환
    table = table.append(pd.DataFrame(results), ignore_index=True)
    print(table)


In [ ]:
# 영상 크롤링
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
urls = pd.read_excel('유튜브영상리스트.xlsx', header=0)

for column in range(len(urls.columns)):
    selected_urls = urls.iloc[:, column]
    
    for url in selected_urls:
        if "shorts" in url:
            short_video(url)
        else:
            general_video(url)
        
print("댓글 데이터프레임:")
print(table)

driver.quit()

In [ ]:
# 10개 더 크롤링
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
urls = pd.read_excel('유튜브영상리스트.xlsx', header=0)

for column in range(8,10):
    selected_urls = urls.iloc[:, column]

    for url in selected_urls:
        if "shorts" in url:
            short_video(url)
        else:
            general_video(url)

print("댓글 데이터프레임:")
print(table)

driver.quit()

In [ ]:
#영어버전 따로 저장
table.to_excel('ENG유튜브영상크롤링DF.xlsx', index=False)

In [ ]:
#기존 테이블과 합치기
pre_table = pd.read_excel('유튜브영상크롤링DF.xlsx', header=0)
combined_table = pd.concat([pre_table, table], ignore_index=True)

print(combined_table)

In [ ]:
#엑셀화
combined_table.to_excel('병합유튜브영상크롤링DF.xlsx', index=False)

# 데이터 전처리

In [ ]:
import pandas as pd

df = pd.read_excel('병합유튜브영상크롤링DF.xlsx')
print(df['comment'])

In [ ]:
###패키지 다운로드
import nltk

# 필수 리소스 목록
resources = ['wordnet', 'stopwords', 'punkt', 'omw-1.4']

for resource in resources:
    try:
        # 리소스가 이미 있는지 확인 (실제 로드를 시도하지 않고 경로만 확인)
        nltk.data.find(f'corpora/{resource}' if resource in ['wordnet', 'stopwords', 'omw-1.4'] else f'tokenizers/{resource}')
        print(f"Resource '{resource}' already downloaded.")
    except LookupError:
        print(f"Resource '{resource}' not found. Downloading...")
        nltk.download(resource, quiet=True) # quiet=True는 다운로드 과정을 간결하게 표시
        print(f"Resource '{resource}' downloaded.")
    except Exception as e:
        # nltk.downloader.DownloadFailure 같은 다른 예외 처리
        print(f"Error checking/downloading resource '{resource}': {e}")
        print(f"Attempting download for '{resource}' again.")
        try:
            nltk.download(resource, quiet=True)
            print(f"Resource '{resource}' downloaded successfully after retry.")
        except Exception as e_retry:
            print(f"Failed to download '{resource}' even after retry: {e_retry}")
            print(f"Please try running 'nltk.download(\"{resource}\")' manually in a Python console.")

print("NLTK resource check/download process complete.")

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from langdetect import detect, DetectorFactory, LangDetectException
from konlpy.tag import Okt

In [ ]:
# Seed 설정
DetectorFactory.seed = 0

# 객체 생성
okt = Okt()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Stopword 세트
english_stop_words = set(stopwords.words('english'))
korean_stop_words = set([
    '이', '그', '저', '것', '수', '들', '좀', '더', '등', '때문', '그리고',
    '그러나', '하지만', '또는', '즉', '즉시', '같다', '다시',
    '많이', '많은', '아주', '매우', '조금', '거의', '이제', '우리', '너무',
    '한다', '했다', '하게', '하려', '해서', '하며',
    '하는', '하자', '하기', '하면', '하니까', '하네', '함', '됨', '됨니다',
    '되다', '되어', '되어서', '되면', '됩니다', '자신', '중', '내', '합시다', 
    '니', '가요', '어요', '네요', '군요', '입니다',
    '가', '은', '는', '의', '에', '와', '과', '을', '를', '로', '자', '관련', '\n', '/n'
    '때', '도', '에서', '하다', '있다', '없다', '대해', '않다', '.', '더욱', '끼리', '뭔가',
    '요', '님', '죠', '네', '어', '께', '데', '딘', '돗', '게', '께서', '합니다',
    '진짜', '정말', '그냥', '뭐', '하나', '처럼', '보기', '까지', '거', '건', '되', '듯',
    '한테', '라니', '그래도', '약간', '또한', '이런', '저런', 'ㅋ', '이다'
])

# 전처리 함수
def clean_text(comment):
    if pd.isna(comment) or len(comment.strip()) == 0:
        return ''
    
    try:
        language = detect(comment)
    except LangDetectException:
        return ''
    
    # 문장 -> 단어 토큰화, 특수문자 제거, 소문자화
    def preprocess_word(word):
        word = word.lower()
        word = re.sub(r'[^\w\s]', '', word)  # 특수문자 제거
        return word

    if language == 'en':
        sentences = sent_tokenize(comment)
        words = []
        for sent in sentences:
            tokens = word_tokenize(sent)
            for word in tokens:
                word = preprocess_word(word)
                if len(word) < 3 or word in english_stop_words:
                    continue
                # lemmatize (동사, 명사 우선)
                lemma = lemmatizer.lemmatize(word, pos='v')
                lemma = lemmatizer.lemmatize(lemma, pos='n')
                stemmed = stemmer.stem(lemma)
                words.append(stemmed)
        return ' '.join(words)
    
    elif language == 'ko':
        desired_pos = ['Noun', 'Adjective']
        tokens = okt.pos(comment, norm=True, stem=True) # 품사 태깅 및 정규화, 어간 추출
        words = []
        for word, pos in tokens:
            if pos in desired_pos: # 원하는 품사만 필터링
                word = preprocess_word(word)
                if len(word) < 2 or word in korean_stop_words:
                    continue
                words.append(word)
        return ' '.join(words)
    
    else:
        return ''

# 적용
df['cleaned_text'] = df['comment'].apply(clean_text)

# 결과 확인
print(df[['comment', 'cleaned_text']].head())

In [ ]:
#테스트 결과 엑셀로 간단 정리
test_df = df[['comment', 'cleaned_text']]

test_df.to_excel('댓글전처리결과.xlsx', index=False)

In [ ]:
import pandas as pd
df = pd.read_excel('댓글전처리결과.xlsx')
df2 = pd.read_excel('병합유튜브영상크롤링DF.xlsx')

df2['cleaned_text'] = df['cleaned_text']
print(df2.head())

# LDA 기반 LDM

In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
import matplotlib.pyplot as plt
import os

df1 = pd.read_excel('댓글전처리결과.xlsx')
df = pd.read_excel('병합유튜브영상크롤링DF.xlsx')

df['cleaned_text'] = df1['cleaned_text']
print(df.head())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# 한글 폰트 설정
matplotlib.rcParams['font.family'] = 'AppleGothic'
matplotlib.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [ ]:
#14개까지 했을 때 
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel
import pandas as pd
import matplotlib.pyplot as plt
import os

# 결과 저장용 리스트
all_results = []
summary_results = []

# 영상(title)별로 그룹화
for vid, group in df.groupby('title'):
    print(f"\n영상 제목: {vid} ===")

    # 전처리
    group = group[group['cleaned_text'].notnull()]
    group = group[group['cleaned_text'].apply(lambda x: isinstance(x, str))]
    documents = group['cleaned_text'].apply(lambda x: x.split()).tolist()

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    coherence_scores = []
    perplexity_scores = []
    topic_nums = list(range(2, 14))

    best_coherence = -1
    best_model = None
    best_num_topics = 0
    best_perplexity = None

    for num_topics in topic_nums:
        lda_model = LdaModel(corpus=corpus, id2word=dictionary,
                             num_topics=num_topics, passes=5, random_state=42)

        coherence_model = CoherenceModel(model=lda_model, texts=documents,
                                         dictionary=dictionary, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        perplexity = lda_model.log_perplexity(corpus)

        coherence_scores.append(coherence_score)
        perplexity_scores.append(perplexity)

        all_results.append({
            'title': vid,
            'num_topics': num_topics,
            'coherence': coherence_score,
            'perplexity': perplexity
        })
        
        print(f"토픽 {num_topics}개 - Coherence: {coherence_score:.4f}, Perplexity: {perplexity:.4f}")

        if coherence_score > best_coherence:
            best_coherence = coherence_score
            best_model = lda_model
            best_num_topics = num_topics
            best_perplexity = perplexity

    # 그래프 1: Coherence
    plt.figure(figsize=(8, 4))
    plt.plot(topic_nums, coherence_scores, marker='o', color='blue')
    plt.title(f"[Coherence] {vid}")
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence Score")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 그래프 2: Perplexity
    plt.figure(figsize=(8, 4))
    plt.plot(topic_nums, perplexity_scores, marker='s', color='red')
    plt.title(f"[Perplexity] {vid}")
    plt.xlabel("Number of Topics")
    plt.ylabel("Perplexity (log)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # 최적 모델 결과 출력 및 저장
    print(f"\n최적 토픽 수: {best_num_topics} (Coherence: {best_coherence:.4f}, Perplexity: {best_perplexity:.4f})")
    topic_list = []
    for idx, topic in best_model.print_topics(num_words=5):
        print(f"토픽 {idx + 1}: {topic}")
        topic_list.append(f"토픽 {idx + 1}: {topic}")

    summary_results.append({
        'title': vid,
        'best_num_topics': best_num_topics,
        'best_coherence': best_coherence,
        'best_perplexity': best_perplexity,
        'topics': "\n".join(topic_list)
    })

# 엑셀 저장
result_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame(summary_results)

with pd.ExcelWriter("토픽분석_결과정리.xlsx", engine="xlsxwriter") as writer:
    result_df.to_excel(writer, sheet_name="전체 토픽별 기록", index=False)
    summary_df.to_excel(writer, sheet_name="최적 토픽 요약", index=False)

print("결과 기록 저장 완료")


### Hyper parameter tuning

In [ ]:
##################
## 9개로 토픽 고정 ##
##################

import seaborn as sns
from itertools import islice

# 실험할 하이퍼파라미터 조합
passes_list = [10, 15, 20]
iterations_list = [50, 100, 150]

# 결과 저장용 리스트
all_results = []
summary_results = []

# 영상(title)별로 그룹화 후 첫 10개만 처리
for vid, group in islice(df.groupby('title'), 10):
    print(f"\n영상 제목: {vid}")

    # 전처리
    group = group[group['cleaned_text'].notnull()]
    group = group[group['cleaned_text'].apply(lambda x: isinstance(x, str))]

    # 여기에 _x000d_ 문자 제거 추가
    group['cleaned_text'] = group['cleaned_text'].str.replace('_x000d_', '')
    group['cleaned_text'] = group['cleaned_text'].str.strip() # 공백 제거

    documents = group['cleaned_text'].apply(lambda x: x.split()).tolist()

    # 빈 문서 필터링 (documents가 비어있는 경우 오류 방지)
    documents = [doc for doc in documents if doc]
    if not documents:
        print(f"[{vid}] 처리할 문서가 없습니다. 다음 영상으로 넘어갑니다.")
        continue

    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    num_topics = 9 # 토픽 수 고정
    best_overall_score = -1
    best_overall_model = None
    best_params = {}

    for passes in passes_list:
        for iterations in iterations_list:
            lda_model = LdaModel(corpus=corpus, id2word=dictionary,
                                    num_topics=num_topics, passes=passes,
                                    iterations=iterations, random_state=42)

            coherence_model = CoherenceModel(model=lda_model, texts=documents,
                                                dictionary=dictionary, coherence='c_v')
            coherence_score = coherence_model.get_coherence()
            perplexity = lda_model.log_perplexity(corpus)

            all_results.append({
                'title': vid,
                'num_topics': num_topics,
                'passes': passes,
                'iterations': iterations,
                'coherence': coherence_score,
                'perplexity': perplexity
            })

            print(f"[{vid}] 토픽 {num_topics}개 | passes={passes}, iterations={iterations} -> "
                    f"Coherence: {coherence_score:.4f}, Perplexity: {perplexity:.4f}")

            if coherence_score > best_overall_score:
                best_overall_score = coherence_score
                best_overall_model = lda_model
                best_params = {
                    'num_topics': num_topics,
                    'passes': passes,
                    'iterations': iterations,
                    'coherence': coherence_score,
                    'perplexity': perplexity
                }

    # 최적 모델 결과 요약
    print(f"\n최적 설정 ({vid}):")
    print(f"토픽 수: {best_params['num_topics']}, passes: {best_params['passes']}, "
          f"iterations: {best_params['iterations']}")
    print(f"Coherence: {best_params['coherence']:.4f}, Perplexity: {best_params['perplexity']:.4f}")

    topic_list = []
    # best_overall_model이 None이 아닐 경우에만 print_topics 호출
    if best_overall_model:
        for idx, topic in best_overall_model.print_topics(num_words=5):
            print(f"토픽 {idx + 1}: {topic}")
            topic_list.append(f"토픽 {idx + 1}: {topic}")
    else:
        print(f"[{vid}] 최적 모델이 생성되지 않았습니다.")


    summary_results.append({
        'title': vid,
        **best_params,
        'topics': "\n".join(topic_list)
    })

    # 시각화를 위한 DataFrame 생성
    # 해당 vid에 대한 결과만 필터링하여 heatmap_data 만들기
    vid_results = [r for r in all_results if r['title'] == vid and r['num_topics'] == num_topics]

    heatmap_data_coherence = pd.DataFrame(index=passes_list, columns=iterations_list, dtype=float)
    heatmap_data_perplexity = pd.DataFrame(index=passes_list, columns=iterations_list, dtype=float)

    for record in vid_results:
        heatmap_data_coherence.loc[record['passes'], record['iterations']] = record['coherence']
        heatmap_data_perplexity.loc[record['passes'], record['iterations']] = record['perplexity']

    # 그래프 1: Coherence 히트맵
    plt.figure(figsize=(10, 6))
    sns.heatmap(heatmap_data_coherence, annot=True, fmt=".4f", cmap="viridis", linewidths=.5)
    plt.title(f"Coherence Score Heatmap for {vid} (Num Topics: {num_topics})")
    plt.xlabel("Iterations")
    plt.ylabel("Passes")
    plt.tight_layout()
    plt.show()

    # 그래프 2: Perplexity 히트맵
    plt.figure(figsize=(10, 6))
    sns.heatmap(heatmap_data_perplexity, annot=True, fmt=".4f", cmap="magma_r", linewidths=.5) # perplexity는 낮을수록 좋으므로 _r (reverse) 사용
    plt.title(f"Perplexity Heatmap for {vid} (Num Topics: {num_topics})")
    plt.xlabel("Iterations")
    plt.ylabel("Passes")
    plt.tight_layout()
    plt.show()

    # 최적 모델 결과 출력 (여기서는 best_params 사용)
    print(f"\n최적 반복 수: (Coherence: {best_params['coherence']:.4f}, Perplexity: {best_params['perplexity']:.4f})")
    print(f"최적 Passes: {best_params['passes']}, 최적 Iterations: {best_params['iterations']}")
    print("최적 모델의 토픽:")
    topic_list = []
    if best_overall_model: # best_overall_model이 None이 아닐 경우에만 print_topics 호출
        for idx, topic in best_overall_model.print_topics(num_words=5):
            print(f"토픽 {idx + 1}: {topic}")
            topic_list.append(f"토픽 {idx + 1}: {topic}")

### passes=20, iterations=100

In [ ]:
import seaborn as sns
from itertools import islice

# 결과 저장용 리스트
all_results = []
summary_results = []

# 토픽 수 탐색 범위
num_topics_list = list(range(2, 16)) # 2부터 15까지

# 영상(title)별로 그룹화
for vid, group in df.groupby('title'):
    print(f"\n영상 제목: {vid}")

    # 전처리
    group = group[group['cleaned_text'].notnull()]
    group['cleaned_text'] = group['cleaned_text'].str.replace('_x000d_', '')
    documents = group['cleaned_text'].apply(lambda x: x.split()).tolist()

    # 빈 문서 필터링 (documents가 비어있는 경우 오류 방지)
    documents = [doc for doc in documents if doc]
    if not documents:
        print(f"[{vid}] 처리할 문서가 없습니다. 다음 영상으로 넘어갑니다.")
        continue
    
    # dictionary와 corpus는 각 영상/문서에 대해 새로 생성
    dictionary = corpora.Dictionary(documents)
    
    # 너무 적은 단어는 필터링 (선택 사항, 모델 품질 향상에 도움)
    dictionary.filter_extremes(no_below=5, no_above=0.5) # 5개 미만 문서에 나타난 단어, 50% 이상 문서에 나타난 단어 제거
    
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    # 코퍼스가 비어있는 경우 (단어 필터링 후) 오류 방지
    if not corpus:
        print(f"[{vid}] 필터링 후 코퍼스가 비어있습니다. 다음 영상으로 넘어갑니다.")
        continue

    coherence_scores = []
    perplexity_scores = []
    
    best_coherence = -1 # 가장 높은 Coherence Score
    best_model = None # 가장 높은 Coherence Score를 달성한 모델
    best_num_topics_for_vid = 0 # 해당 영상의 최적 토픽 수
    best_perplexity_for_vid = None # 해당 영상의 최적 Perplexity

    # num_topics_list (2~14개)를 순회하며 모델 학습 및 평가
    for num_topic_candidate in num_topics_list:
        try:
            lda_model = LdaModel(corpus=corpus, id2word=dictionary,
                                 num_topics=num_topic_candidate,
                                 passes=20,
                                 iterations=100, 
                                 random_state=42)

            coherence_model = CoherenceModel(model=lda_model, texts=documents,
                                             dictionary=dictionary, coherence='c_v')
            coherence_score = coherence_model.get_coherence()
            perplexity = lda_model.log_perplexity(corpus)

            coherence_scores.append(coherence_score)
            perplexity_scores.append(perplexity)

            # 각 num_topic_candidate에 대한 결과 저장
            all_results.append({
                'title': vid,
                'num_topics': num_topic_candidate,
                'coherence': coherence_score,
                'perplexity': perplexity
            })
            
            print(f"[{vid}] 토픽 {num_topic_candidate}개 - Coherence: {coherence_score:.4f}, Perplexity: {perplexity:.4f}")

            # 현재 영상의 최적 토픽 수 찾기 (Coherence 기반)
            if coherence_score > best_coherence:
                best_coherence = coherence_score
                best_model = lda_model
                best_num_topics_for_vid = num_topic_candidate
                best_perplexity_for_vid = perplexity
        
        except Exception as e:
            print(f"[{vid}] 토픽 {num_topic_candidate}개 모델 학습 중 오류 발생: {e}")
            # 오류 발생 시 해당 토픽 수에 대한 결과는 추가하지 않음
            coherence_scores.append(0.0) # 또는 NaN
            perplexity_scores.append(0.0) # 또는 NaN


    # 각 영상별 그래프 1: Coherence
    if coherence_scores: # 데이터가 있을 때만 그래프 그리기
        plt.figure(figsize=(8, 4))
        plt.plot(num_topics_list[:len(coherence_scores)], coherence_scores, marker='o', color='blue')
        plt.title(f"[Coherence] {vid}")
        plt.xlabel("Number of Topics")
        plt.ylabel("Coherence Score")
        plt.grid(True)
        plt.xticks(num_topics_list[:len(coherence_scores)]) # x축 레이블 설정
        plt.tight_layout()
        plt.show()

    # 각 영상별 그래프 2: Perplexity
    if perplexity_scores: # 데이터가 있을 때만 그래프 그리기
        plt.figure(figsize=(8, 4))
        plt.plot(num_topics_list[:len(perplexity_scores)], perplexity_scores, marker='s', color='red')
        plt.title(f"[Perplexity] {vid}")
        plt.xlabel("Number of Topics")
        plt.ylabel("Perplexity (log)")
        plt.grid(True)
        plt.xticks(num_topics_list[:len(perplexity_scores)]) # x축 레이블 설정
        plt.tight_layout()
        plt.show()

    # 최적 모델 결과 출력 및 저장
    if best_model: # best_model이 존재하는 경우에만
        print(f"\n최적 토픽 수 ({vid}): {best_num_topics_for_vid} "
              f"(Coherence: {best_coherence:.4f}, Perplexity: {best_perplexity_for_vid:.4f})")
        topic_list = []
        for idx, topic in best_model.print_topics(num_words=5):
            print(f"토픽 {idx + 1}: {topic}")
            topic_list.append(f"토픽 {idx + 1}: {topic}")

        summary_results.append({
            'title': vid,
            'best_num_topics': best_num_topics_for_vid,
            'best_coherence': best_coherence,
            'best_perplexity': best_perplexity_for_vid,
            'topics': "\n".join(topic_list)
        })
    else:
        print(f"\n[{vid}] 최적 모델을 찾을 수 없거나 데이터가 부족합니다.")

In [ ]:
# 엑셀 저장
result_df = pd.DataFrame(all_results)
summary_df = pd.DataFrame(summary_results)

file_path = "토픽 분석 결과.xlsx"

with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
    result_df.to_excel(writer, sheet_name="전체 토픽별 기록", index=False)
    summary_df.to_excel(writer, sheet_name="최적 토픽 요약", index=False)

print(f"\n결과 기록 저장 완료: {file_path}")

### 그래프그리기

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import io

excel_file_path = "토픽 분석 결과.xlsx"
sheet_name = "최적 토픽 요약"

df = pd.read_excel(excel_file_path, sheet_name=sheet_name)


In [ ]:
coherence_col = 'best_coherence'
perplexity_col = 'best_perplexity'

for row in df:
    #coherence
    plt.figure(figsize=(8, 4))
    plt.plot(coherence_scores, marker='o', color='blue')
    plt.title(f"[Coherence] ", row[0])
    plt.xlabel("Number of Topics")
    plt.ylabel("Coherence Score")
    plt.grid(True)
    plt.xticks(num_topics_list[:len(coherence_scores)]) # x축 레이블 설정
    plt.tight_layout()
    plt.show()

    #perplexity
    plt.figure(figsize=(8, 4))
    plt.plot(num_topics_list[:len(perplexity_scores)], perplexity_scores, marker='s', color='red')
    plt.title(f"[Perplexity] ", row[0])
    plt.xlabel("Number of Topics")
    plt.ylabel("Perplexity (log)")
    plt.grid(True)
    plt.xticks(num_topics_list[:len(perplexity_scores)]) # x축 레이블 설정
    plt.tight_layout()
    plt.show()
